In [ ]:
# This notebook consists of actions for preparation for `SeismicGeometry` tests
# It creates a fake cube as random noise and convert it into available data formats
import os
import sys
import shutil
import warnings
warnings.filterwarnings('ignore')

import numpy as np

sys.path.insert(0, '../../../seismiqb')

from seismiqb import SeismicGeometry
from seismiqb.geometry import export

In [ ]:
# Data creation parameters
CUBE_SHAPE = (1000, 200, 400)
SEED = 42

# Data formats for which running tests
FORMATS = ['sgy', 'hdf5', 'qhdf5', 'blosc', 'qblosc']

# Prepare a workspace

Create directories for files and create a fake cube and save it.

**Storage structure:**
___


**geometry_test_files** (geometry tests directory with temporary files)

&emsp;├── **test_array.npy**

&emsp;└── **test_cube.{DATAFORMAT}**

---
 * **{DATAFORMAT}** means each data format from **FORMATS** list

In [ ]:
OUTPUT_DIR = './geometry_test_files'

# (Re)create the test directory
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(OUTPUT_DIR)

# Create a fake cube

In [ ]:
CUBE_PATH = os.path.join(OUTPUT_DIR, 'test_cube.sgy')

rng = np.random.default_rng(SEED)
data_array = rng.normal(0, 1000, CUBE_SHAPE).astype(np.float32)

with open(os.path.join(OUTPUT_DIR, 'test_array.npy'), 'wb') as outfile:
    np.save(outfile, data_array)

In [ ]:
%%time
export.make_segy_from_array(array=data_array, path_segy=CUBE_PATH, zip_segy=False,
                            sample_rate=2., delay=50, pbar='t')

In [ ]:
%%time
geometry_sgy = SeismicGeometry(path=CUBE_PATH,
                               process=True, collect_stats=True, spatial=True,
                               pbar='t')

loading_error = "Saved and loaded sgy geometry files are different"
assert np.allclose(data_array, geometry_sgy[:, :, :]), loading_error

# Check data loading

In [ ]:
%%time
geometry_sgy = SeismicGeometry(path=CUBE_PATH,
                               headers=SeismicGeometry.HEADERS_POST_FULL,
                               index_headers=SeismicGeometry.INDEX_CDP)

In [ ]:
%%time
geometry_sgy = SeismicGeometry(path=CUBE_PATH,
                               headers=SeismicGeometry.HEADERS_POST_FULL,
                               index_headers=SeismicGeometry.INDEX_POST)

assert np.allclose(data_array, geometry_sgy[:, :, :]), loading_error

# Conversion

In [ ]:
for data_format in FORMATS:
    if data_format != 'sgy':
        _ = geometry_sgy.convert(format=data_format, quantize=False, store_meta=False, pbar='t')

# Create new sgy from array with spec from the existed sgy

In [ ]:
NEW_CUBE_PATH = os.path.join(OUTPUT_DIR, 'test_new_cube.sgy')

array = np.random.normal(size=geometry_sgy.shape)
export.make_segy_from_array(array, NEW_CUBE_PATH, zip_segy=False, path_spec=CUBE_PATH)
new_geometry_sgy = SeismicGeometry(NEW_CUBE_PATH, collect_stats=False)

In [ ]:
assert geometry_sgy.ilines_offset == new_geometry_sgy.ilines_offset, 'Wrong ilines offset'
assert geometry_sgy.xlines_offset == new_geometry_sgy.xlines_offset, 'Wrong xlines offset'
assert (new_geometry_sgy.dataframe == geometry_sgy.dataframe).all().all(), 'Different geometries'

Creation with origin

In [ ]:
array = np.random.normal(size=(20, 20, 20))
export.make_segy_from_array(array, NEW_CUBE_PATH, origin=(5, 10, 15), zip_segy=False, path_spec=CUBE_PATH)
new_geometry_sgy = SeismicGeometry(NEW_CUBE_PATH, collect_stats=False)

assert geometry_sgy.ilines_offset + 5 == new_geometry_sgy.ilines_offset, 'Wrong ilines offset'
assert geometry_sgy.xlines_offset + 10 == new_geometry_sgy.xlines_offset, 'Wrong xlines offset'

In [ ]:
df = geometry_sgy.dataframe.loc[new_geometry_sgy.dataframe.index][['CDP_X', 'CDP_Y']]
new_df = new_geometry_sgy.dataframe[['CDP_X', 'CDP_Y']]

assert (df == new_df).all().all()